In [ ]:
'''03_create_mysql_debezium_connector'''

'''--------------------------------------------------------------------------------------'''

'''kita akan membuat connector dari ubuntu yang terinstall di dalam hyper-v
ke mysql yang terinstall di dalam docker container'''

'''check kafka topic yang sudah ada'''
kafka-topics --list --bootstrap-server localhost:9092

'''debezium connector akan membuat topic secara otomatis, maka kita tidak perlu membuat topic secara manual'''

'''--------------------------------------------------------------------------------------'''

'''kita akan membuat sebuah connector dari mysql ke kafka'''

curl -i -X POST -H "Accept:application/json" -H "Content-Type:application/json" http://localhost:8083/connectors/ -d '
{
    "name": "mysql-source-connector",
    "config": {
        "connector.class": "io.debezium.connector.mysql.MySqlConnector",
        "tasks.max": "1",
        "database.hostname": "nb633.elife.co.id",
        "database.port": "3307",
        "database.user": "root",
        "database.password": "password",
        "database.server.id": "100001",
        "database.server.name": "mysql_source",
        "database.include.list": "replication",
        "database.history.kafka.bootstrap.servers": "localhost:9092",
        "database.history.kafka.topic": "PLAIN",
        "key.converter": "io.confluent.connect.avro.AvroConverter",
        "value.converter": "io.confluent.connect.avro.AvroConverter",
        "key.converter.schema.registry.url": "http://localhost:8081",
        "value.converter.schema.registry.url": "http://localhost:8081",
        "topic.prefix": "mysql_source",
        "schema.history.internal.kafka.bootstrap.servers": "localhost:9092",
        "schema.history.internal.kafka.topic": "PLAIN"
    }
}'

'''--------------------------------------------------------------------------------------'''

'''jika berhasil maka akan muncul hasil berikut'''
HTTP/1.1 201 Created
Date: Sun, 19 Nov 2023 15:23:18 GMT
Location: http://localhost:8083/connectors/mysql-source-connector
Content-Type: application/json
Content-Length: 802

''' dan seterusnya '''

'''--------------------------------------------------------------------------------------'''

'''connector akan membuat topic secara otomatis, kita perlu check apakah topic sudah terbuat atau belum'''
kafka-topics --list --bootstrap-server localhost:9092

'''--------------------------------------------------------------------------------------'''

'''jika ada yang salah, maka kita dapat menghapus connector tersebut dengan perintah berikut:'''
curl -X DELETE http://localhost:8083/connectors/mysql-source-connector

''jangan lupa untuk menghapus topic dari connector yang dihapus, caranya ketik perintah berikut:''
kafka-topics --delete --bootstrap-server localhost:9092 --topic mysql_source.replication.user

'''--------------------------------------------------------------------------------------'''

'''sekarang kita check apakah connector sudah terinstall atau belum'''
curl -XGET http://localhost:8083/connectors | jq

'''maka akan muncul hasil berikut'''
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    26  100    26    0     0   5891      0 --:--:-- --:--:-- --:--:--  6500
[
  "mysql-source-connector"
]

'''--------------------------------------------------------------------------------------'''

'''untuk dapat memonitoring connector yang sudah terinstall, kita dapat menggunakan perintah berikut'''
curl -XGET http://localhost:8083/connectors?expand=status | jq

'''maka akan muncul response sebagai berikut:'''
{
  "mysql-source-connector": {
    "status": {
      "name": "mysql-source-connector",
      "connector": {
        "state": "RUNNING",
        "worker_id": "127.0.1.1:8083"
      },
      "tasks": [
        {
          "id": 0,
          "state": "RUNNING",
          "worker_id": "127.0.1.1:8083"
        }
      ],
      "type": "source"
    }
  }
}

'''--------------------------------------------------------------------------------------'''

'''untuk melihat metadata dari connector yang sudah terinstall, kita dapat menggunakan perintah berikut'''
curl -XGET http://localhost:8083/connectors/mysql-source-connector?expand=info | jq

'''maka akan muncul response sebagai berikut:'''
{
  "name": "mysql-source-connector",
  "config": {
    "connector.class": "io.debezium.connector.mysql.MySqlConnector",
    "database.user": "root",
    "database.server.id": "100001",
    "tasks.max": "1",
    "database.history.kafka.bootstrap.servers": "localhost:9092",
    "database.history.kafka.topic": "PLAIN",
    "database.server.name": "mysql_source",
    "schema.history.internal.kafka.bootstrap.servers": "localhost:9092",
    "database.port": "3307",
    "value.converter.schema.registry.url": "http://localhost:8081",
    "topic.prefix": "mysql_source",
    "schema.history.internal.kafka.topic": "PLAIN",
    "database.hostname": "nb633.elife.co.id",
    "database.password": "password",
    "name": "mysql-source-connector",
    "value.converter": "io.confluent.connect.avro.AvroConverter",
    "key.converter": "io.confluent.connect.avro.AvroConverter",
    "key.converter.schema.registry.url": "http://localhost:8081",
    "database.include.list": "replication"
  },
  "tasks": [
    {
      "connector": "mysql-source-connector",
      "task": 0
    }
  ],
  "type": "source"
}

'''--------------------------------------------------------------------------------------'''

'''sekarang saatnya kita coba mengubah data di mysql dan melihat apakah data tersebut masuk ke kafka topic atau tidak'''

'''pertama-tama pastikan lagi bahwa CONFLUENT_HOME sudah di set dan
CONFLUENT_HOME/bin sudah di set ke dalam PATH'''

'''ketikkan perintah berikut untuk melihat apakah PATH sudah di set atau belum'''
echo $PATH
/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/lib/jvm/java-11-openjdk-amd64/bin:/opt/confluent/confluent-7.5.2/bin

'''ketikkan perintah berikut untuk melihat apakah CONFLUENT_HOME sudah di set atau belum'''
echo $CONFLUENT_HOME
/opt/confluent/confluent-7.5.2

'''$CONFLUENT_HOME dan $PATH sudah di set, maka kita dapat melanjutkan ke tahap berikutnya'''

'''--------------------------------------------------------------------------------------'''

'''jalankan command di bawah ini untuk memastikan apakah kafka topic sudah terbuat atau belum'''
kafka-topics --list --bootstrap-server localhost:9092

'''jika topics mysql_source dan mysql_source.replication.user sudah terbuat, maka kita dapat melanjutkan ke tahap berikutnya'''
mysql_source
mysql_source.replication.user

'''--------------------------------------------------------------------------------------'''

'''sekarang kita akan memonitor perubahan yang tertangkap oleh connector, 
menggunakan console consumer dari confluent'''

'''ketikkan perintah berikut untuk memonitor perubahan yang tertangkap oleh connector'''
kafka-avro-console-consumer --bootstrap-server localhost:9092 --topic mysql_source.replication.user --from-beginning --property schema.registry.url=http://localhost:8081

'''--------------------------------------------------------------------------------------'''

'''maka akan muncul isi dari table user yang ada di mysql_source'''
{"before":null,"after":{"mysql_source.replication.user.Value":{"user_id":1,"first_name":{"string":"John"},"last_name":{"string":"Doe"},"city":{"string":"New York"},"state":{"string":"NY"},"pincode":{"string":"10001"}}},"source":{"version":"2.4.1.Final","connector":"mysql","name":"mysql_source","ts_ms":1700422755000,"snapshot":{"string":"first"},"db":"replication","sequence":null,"table":{"string":"user"},"server_id":0,"gtid":null,"file":"binlog.000004","pos":1413,"row":0,"thread":null,"query":null},"op":"r","ts_ms":{"long":1700422760453},"transaction":null}
{"before":null,"after":{"mysql_source.replication.user.Value":{"user_id":2,"first_name":{"string":"Iman"},"last_name":{"string":"Subarkah"},"city":{"string":"Los Angeles"},"state":{"string":"LA"},"pincode":{"string":"10002"}}},"source":{"version":"2.4.1.Final","connector":"mysql","name":"mysql_source","ts_ms":1700422755000,"snapshot":{"string":"last"},"db":"replication","sequence":null,"table":{"string":"user"},"server_id":0,"gtid":null,"file":"binlog.000004","pos":1413,"row":0,"thread":null,"query":null},"op":"r","ts_ms":{"long":1700422760453},"transaction":null}
"

'''--------------------------------------------------------------------------------------'''

'''masuk ke mysql yang ada di docker container'''
mysql -h nb633.elife.co.id -u root -p -P 3307

'''--------------------------------------------------------------------------------------'''

'''test insert user baru'''
insert into user (first_name, last_name, city, state, pincode) values ('Achmad', 'Taufan', 'Bekasi', 'BK', '10003');

'''hasilnya di consumer'''
{"before":null,
"after":{
  "mysql_source.replication.user.Value":{
    "user_id":3,
    "first_name":{"string":"Achmad"},
    "last_name":{"string":"Taufan"},
    "city":{"string":"Bekasi"},
    "pincode":{"string":"10002"}
  }
},"source":{"version":"2.4.1.Final","connector":"mysql","name":"mysql_source","ts_ms":1700422755000,"snapshot":{"string":"last"},"db":"replication","sequence":null,"table":{"string":"user"},"server_id":0,"gtid":null,"file":"binlog.000004","pos":1413,"row":0,"thread":null,"query":null},"op":"r","ts_ms":{"long":1700422760453},"transaction":null}

'''--------------------------------------------------------------------------------------'''

'''test update user yang sudah ada'''
update user set city = 'Jababeka' where user_id = 3;

'''hasilnya di consumer'''
{"before":{
  "mysql_source.replication.user.Value":{
    "user_id":3,
    "first_name":{"string":"Achmad"},
    "last_name":{"string":"Taufan"},
    "city":{"string":"Bekasi"},
    "state":{"string":"BK"},
    "pincode":{"string":"10003"}
  }
},
"after":{
  "mysql_source.replication.user.Value":{
    "user_id":3,
    "first_name":{"string":"Achmad"},
    "last_name":{"string":"Taufan"},
    "city":{"string":"Jababeka"},
    "state":{"string":"BK"},
    "pincode":{"string":"10003"}
  }
},"source":{"version":"2.4.1.Final","connector":"mysql","name":"mysql_source","ts_ms":1700422755000,"snapshot":{"string":"false"},"db":"replication","sequence":null,"table":{"string":"user"},"server_id":0,"gtid":null,"file":"binlog.000004","pos":1413,"row":0,"thread":null,"query":null},"op":"u","ts_ms":{"long":1700422760453},"transaction":null}

'''--------------------------------------------------------------------------------------'''

'''test delete user yang sudah ada'''
delete from user where user_id = 3;

'''hasilnya di consumer'''
{"before":{
  "mysql_source.replication.user.Value":{
    "user_id":3,
    "first_name":{"string":"Achmad"},
    "last_name":{"string":"Taufan"},
    "city":{"string":"Jababeka"},
    "state":{"string":"BK"},
    "pincode":{"string":"10003"}
  }
},"after":null,"source":{"version":"2.4.1.Final","connector":"mysql","name":"mysql_source","ts_ms":1700440225000,"snapshot":{"string":"false"},"db":"replication","sequence":null,"table":{"string":"user"},"server_id":100001,"gtid":null,"file":"binlog.000004","pos":2376,"row":0,"thread":{"long":281},"query":null},"op":"d","ts_ms":{"long":1700440230955},"transaction":null}

'''--------------------------------------------------------------------------------------'''
